In [1]:
import numpy as np
from torch.utils.data import IterableDataset
from datetime import datetime, time, timedelta
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import xarray as xr
from ocf_blosc2 import Blosc2
from datetime import timezone

import json
from time import sleep
from IPython.display import Video


In [2]:
def month_nonhrv_getter(year, month):
    nonhrv = xr.open_dataset(
        f"/data/climatehack/official_dataset/nonhrv/{year}/{month}.zarr.zip",
        engine="zarr",
        consolidated=True,
    )
    def get_nonhrv_at_time(date):
        return nonhrv["data"].sel(time=date).to_numpy()

    return get_nonhrv_at_time

def month_weather_getter(year, month):
    weather = xr.open_dataset(
        f"/data/climatehack/official_dataset/weather/{year}/{month}.zarr.zip",
        engine="zarr",
        consolidated=True,
    )
    data_vars = np.array(list(weather.data_vars.keys()))
    def get_weather_at_time(date):
        out = [weather[data_var].sel(time=date) for data_var in data_vars]  #maybe subtract 273.15 because Kelvin!!! (only for temp tho)
        out = np.stack(out, axis=-1)
        return out

    return get_weather_at_time

# year, month, day, hour, minute = 2020, 5, 19, 4, 0
# d = datetime(year, month, day, hour, minute)
# month_weather_getter(2020, 5)(d)

In [3]:
with open("../indices.json") as f:
	site_locations = {
		data_source: {
			int(site): (int(location[0]), int(location[1]))
			for site, location in locations.items()
                        }
		for data_source, locations in json.load(f).items()
                 }

In [4]:
def month_pv_getter(year, month):
    pv = pd.read_parquet(f"/data/climatehack/official_dataset/pv/{year}/{month}.parquet").drop("generation_wh", axis=1)
    def get_pv_at_time(date, ss_id = "all"):
        return pv.loc[date]
    return get_pv_at_time

In [11]:
from matplotlib import pyplot as plt 
import numpy as np 
import matplotlib.animation as animation 
from IPython import display 
  
def makeVideo(START_DATE_PV, CHANNEL=8, MINUTES_INCR=60, DATASET="nonhrv", FRAMES=10, SAVENAME="test.html"):
    assert START_DATE_PV.hour == 4, "start hour not 4"
    d = START_DATE_PV
    START_DATE_NONHRV = datetime(d.year, d.month, d.day, d.hour, d.minute)
    get_pv_at_time = month_pv_getter(year, month)
    
    get_image_at_time = month_nonhrv_getter(year, month) if DATASET=="nonhrv" else month_weather_getter(year, month)

    site = site_locations[DATASET]
    pvs = get_pv_at_time(START_DATE_PV, ss_id = "all")
    # x, y = [site[key][0] for key in site if key in pvs.index], [site[key][1] for key in site if key in pvs.index]
    # c = list(pvs["power"])
    hours_of_data = 13 #we look from 4:00 to 17:00

    fig = plt.figure() 
    
    axis = plt.imshow(get_image_at_time(START_DATE_NONHRV)[:,:,CHANNEL], cmap="gray", vmin=0, vmax=1)
    
    def animate(frame_number): 
        time_delta = timedelta(minutes = MINUTES_INCR * (frame_number % (hours_of_data * 60 // MINUTES_INCR)) + 24 * 60 * (frame_number // (13 * 60 // MINUTES_INCR)))
        axis.set_data(get_image_at_time(START_DATE_NONHRV + time_delta)[:,:,CHANNEL])

        pvs = get_pv_at_time(START_DATE_PV + time_delta, ss_id = "all")
        site_validate = pvs.index
        x, y = [site[key][0] for key in site_validate], [site[key][1] for key in site_validate]
        c = pvs["power"]
        plt.scatter(x, y, s=10, c=c, cmap='inferno')
        return axis, 
    
    
    anim = animation.FuncAnimation(fig, animate, frames=FRAMES,  
                                interval=MINUTES_INCR / (60*hours_of_data) * 5000, blit=True) 
    
    video = anim.to_html5_video() 
    with open(f"vids/{SAVENAME}", 'w') as f:
        f.write(video)
    html = display.HTML(video) 
    display.display(html) 
    plt.close() 

year, month, day, hour, minute = 2020, 12, 20, 4, 0
START_DATE_PV = datetime(year, month, day, hour, minute, tzinfo=timezone(timedelta(hours=0)))
START_DATE_NONHRV = datetime(year, month, day, hour, minute)
MINUTES_INCR = 15
weather_vars = ['alb_rad', 'aswdifd_s', 'aswdir_s', 'cape_con', 'clch', 'clcl', 'clcm', 'clct',
 'h_snow', 'omega_1000', 'omega_700', 'omega_850', 'omega_950', 'pmsl',
 'relhum_2m', 'runoff_g', 'runoff_s', 't_2m', 't_500', 't_850', 't_950', 't_g',
 'td_2m', 'tot_prec', 'u_10m', 'u_50', 'u_500', 'u_850', 'u_950', 'v_10m', 'v_50',
 'v_500', 'v_850', 'v_950', 'vmax_10m', 'w_snow', 'ww', 'z0']
nonhrv_vars = ["IR_016", "IR_039", "IR_087", "IR_097", "IR_108", "IR_120", "IR_134", "VIS006", "VIS008", "WV_062 ""WV_073"]
DATASET="nonhrv"
CHANNEL = weather_vars.index('clcm') if DATASET=="weather" else nonhrv_vars.index("IR_016")
CHANNEL = 8
for CHANNEL in range(11):
    SAVENAME = f"{DATASET}_{CHANNEL}_{month}_{day}.html"
    makeVideo(START_DATE_PV, CHANNEL=CHANNEL, MINUTES_INCR=15, DATASET=DATASET, FRAMES=100, SAVENAME="test.html")

In [18]:
with open('vids/test.html', 'r') as f:
    content = f.read()
html = display.HTML(content) 
display.display(html) 